<div>
<img src='https://upload.wikimedia.org/wikipedia/commons/6/64/Sharif_University_Logo.jpg' alt="SUT logo" width=220 height=220 align=left class="saturate">

<br>
<font face="Times New Roman">
<div dir=ltr align=center>
<!-- <font color=0F5298 size=7> -->
<font color=0F5298 size=6>
    Introduction to Machine Learning <br> <br>
<!-- <font color=2565AE size=5> -->
<font size=5>
    Computer Engineering Department <br>
    Spring 2023 <br> <br>
<font color=606060 size=5>
    Homeworks 8 & 9 (Combined): Practical - Faces <br> <br>
<font color=686880 size=4>
    TAs: Arman Malekzadeh - Amirhossein Ramazani Bonab - Yalda Shabanzadeh
    
____

### Full Name : Parsa Sharifi
### Student Number : 99101762
### Colab Link: https://colab.research.google.com/drive/1bXSizCYH8ios-R4at2TXMVdXfKQCCx6W?usp=sharing
___

In this assignment, you'll be dealing with a dataset consisting of 400 faces, belonging to 40 people. Your main task is to train models capable of recognizing those faces.
The faces are `jpg` images packed together as a zip file called `images.zip`, and the `meta.csv` determines from which person each image was taken.

In [22]:
!wget -O images.zip https://www.dropbox.com/s/wrias0bjyte3rr2/images.zip?dl=1
!wget -O meta.csv https://www.dropbox.com/s/vqt1v8sfmz18rcf/faces-metadata.csv?dl=1

--2023-06-27 13:32:34--  https://www.dropbox.com/s/wrias0bjyte3rr2/images.zip?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6031:18::a27d:5112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/wrias0bjyte3rr2/images.zip [following]
--2023-06-27 13:32:35--  https://www.dropbox.com/s/dl/wrias0bjyte3rr2/images.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc6da48039e51180d98c8721ba2d.dl.dropboxusercontent.com/cd/0/get/B-ya6d9Wo7vi30RV5JXS3XVjYPvpI2WfMnaVOBAzLNAToau8V7EBOfcQ0EOOCfk3IKMygzGSw3hSmi_qrlx3CLaIoidXy8G8gzx2VOvBqAb-wM2Qpn6hAONNGq39y7YFljDfrbIuL7jYon5ago4yAvo_LN_dbUeeUiyZaXJHsNZ4dUqIe4tJdUvUw-i0XtIRIN8/file?dl=1# [following]
--2023-06-27 13:32:35--  https://uc6da48039e51180d98c8721ba2d.dl.dropboxusercontent.com/cd/0/get/B-ya6d9Wo7vi30RV5JXS3XVjYPvpI2WfMnaVOBAzLNAToau8V7EBOfcQ0EOOCf

In [23]:
# you can use the following packages
from PIL import Image
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.cluster import KMeans
from sklearn.decomposition import TruncatedSVD
import plotly.express as px

# Sec 1: Data Preparation

Load all images as a numpy array (called `x`) and the metadata as a dataframe **(P1-1: 25 points)**

In [24]:
!unzip /content/images.zip

Archive:  /content/images.zip
replace content/img-data/50.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
 extracting: content/img-data/50.jpg  
replace content/img-data/181.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [48]:
metadata = pd.read_csv('meta.csv')

In [110]:
img_paths = metadata['path'].values
images = []
for img_path in img_paths:
    img = Image.open('content/img-data/' +img_path).convert('L')
    img = np.array(img)
    images.append(img)
x = np.array(images)

Flatten and normalize the images **(P1-2: 10 points)**

Note: For this part, normalization only means dividing by 255.

In [111]:
x = x.reshape(x.shape[0], -1)
x = x / 255

Encode the labels using `LabelEncoder` and save the result as a numpy array called `y` **(P1-3: 5 points)**

In [35]:
metadata

,path,label
0,0.jpg,jones
1,1.jpg,jones
2,2.jpg,jones
3,3.jpg,jones
4,4.jpg,jones
...,...,...
395,395.jpg,vaux
396,396.jpg,vaux
397,397.jpg,vaux
398,398.jpg,vaux


In [112]:
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(metadata['label'])

Split the data into training (80%), validation (10%), and testing (10%) parts. **(P1-4: 10 points)**

Note: The split procedure should maintain the class balance. This is sometimes called "[stratified sampling](https://en.wikipedia.org/wiki/Stratified_sampling)".

In [113]:
random_state = 42
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify=y, random_state=random_state)
x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, test_size=0.5, stratify=y_test, random_state=random_state)


# Sec 2: SVM

## Subsec 2.1.

Train a SVM using `sklearn`. Note that the hyperparameter tuning should be done using the "validation" set. **(P2-1-1: 20 points)**

In [122]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

C_values = [.01, 0.1, 1, 10, 100]
gamma_values = ['scale', 'auto']
kernel_range = ['linear', 'rbf', 'poly']
degree_range = [2, 3, 4]

best_accuracy = 0

for c in C_values:
    for kernel in kernel_range:
        for degree in degree_range:
            for gamma in gamma_values:
                svm = SVC(C=c, kernel=kernel, degree=degree, gamma=gamma)
                svm.fit(x_train, y_train)
                y_pred = svm.predict(x_val)
                accuracy = accuracy_score(y_val, y_pred)
                if accuracy > best_accuracy:
                  best_accuracy = accuracy
                  best_C = c
                  best_gamma = gamma
                  best_kernel = kernel
                  best_degree = degree
                  best_params = {'C': c, 'kernel': kernel, 'degree': degree, 'gamma': gamma}

svm = SVC(C=best_C, kernel=best_kernel, degree=best_degree, gamma=best_gamma)
svm.fit(np.concatenate((x_train, x_val)), np.concatenate((y_train, y_val)))

SVC(C=0.01, kernel='poly')

Report the accuracy and F1-score of the classifier on the testing set **(P2-1-2: 10 points)**

In [123]:
y_test_pred = svm.predict(x_test)
test_accuracy = accuracy_score(y_test, y_test_pred)

In [124]:
best_params

{'C': 0.01, 'kernel': 'poly', 'degree': 3, 'gamma': 'scale'}

In [125]:
test_accuracy

0.95

## Subsec 2.2.

Train a SVM from scratch using the following class **(P2-2-1: 40 points)**

Note: For this part, you are not required to use the validation set for hyperparameter tuning. However, you might get better results if you do that.

!!!!Attention:

I implemented 2 codes for binary svm (SVM_bi), first is iterative and second solves the problem using quadratic programming

the SVM class is a one vs one multy class svm tha uses SVM_bi

the second SVM_bi class works better than the first(iterative)

In [ ]:
class SVM_bi:
    def __init__(self, C=None, kernel='linear'):
        # C = error term
        self.C = C
        self.w = 0
        self.b = 0
          # Hinge Loss Function / Calculation
    def hingeloss(self, w, b, x, y):
        # Regularizer term
        reg = 0.5 * (w * w)

        for i in range(x.shape[0]):
            # Optimization term
            opt_term = y[i] * ((np.dot(w, x[i])) + b)

            # calculating loss
            loss = reg + self.C * max(0, 1-opt_term)
        return loss[0][0]
    def fit(self, X, Y, batch_size=100, learning_rate=0.001, epochs=1000):
            # The number of features in X
            number_of_features = X.shape[1]

            # The number of Samples in X
            number_of_samples = X.shape[0]

            c = self.C

            # Creating ids from 0 to number_of_samples - 1
            ids = np.arange(number_of_samples)

            # Shuffling the samples randomly
            np.random.shuffle(ids)

            # creating an array of zeros
            w = np.zeros((1, number_of_features))
            b = 0
            losses = []

            # Gradient Descent logic
            for i in range(epochs):
                # Calculating the Hinge Loss
                l = self.hingeloss(w, b, X, Y)

                # Appending all losses
                losses.append(l)

                # Starting from 0 to the number of samples with batch_size as interval
                for batch_initial in range(0, number_of_samples, batch_size):
                    gradw = 0
                    gradb = 0

                    for j in range(batch_initial, batch_initial + batch_size):
                        if j < number_of_samples:
                            x = ids[j]
                            ti = Y[x] * (np.dot(w, X[x].T) + b)

                            if ti > 1:
                                gradw += 0
                                gradb += 0
                            else:
                                # Calculating the gradients

                                #w.r.t w
                                gradw += c * Y[x] * X[x]
                                # w.r.t b
                                gradb += c * Y[x]

                    # Updating weights and bias
                    w = w - learning_rate * w + learning_rate * gradw
                    b = b + learning_rate * gradb

            self.w = w
            self.b = b

            return self.w, self.b, losses
    def predict(self, X):

      prediction = np.dot(X, self.w[0]) + self.b # w.x + b
      return np.sign(prediction)

In [40]:
class SVM_bi(object):
    def __init__(self, C=None, kernel='linear'):
        if kernel == 'linear':
            self.kernel = lambda x1, x2: np.dot(x1, x2)
        elif kernel == 'poly':
            self.kernel = lambda x1, x2: (np.dot(x1, x2) + 1)**2 # change degree as needed
        elif kernel == 'rbf':
            self.kernel = lambda x1, x2: np.exp(-np.linalg.norm(x1 - x2)**2) # change gamma as needed
        elif kernel == 'sigmoid':
            self.kernel = lambda x1, x2: np.tanh(np.dot(x1, x2)) # change coef0 as needed
        else:
            raise ValueError('!!Invalid kernel')
        self.C = C
        if self.C is not None: self.C = float(self.C)

    def fit(self, X, y):

        n_samples, n_features = X.shape

        K = np.zeros((n_samples, n_samples))
        for i in range(n_samples):
            for j in range(n_samples):
                K[i,j] = self.kernel(X[i], X[j])

        P = cvxopt.matrix(np.outer(y,y) * K)
        q = cvxopt.matrix(np.ones(n_samples) * -1)
        A = cvxopt.matrix(y, (1,n_samples), 'd')
        b = cvxopt.matrix(0.0)
        if self.C == None:
            G = cvxopt.matrix(np.diag(np.ones(n_samples) * -1))
            h = cvxopt.matrix(np.zeros(n_samples))
        else:
            G_1 = np.diag(np.ones(n_samples) * -1)
            G_2 = np.identity(n_samples)
            G = cvxopt.matrix(np.vstack((G_1, G_2)))
            h_1 = np.zeros(n_samples)
            h_2 = np.ones(n_samples) * self.C
            h = cvxopt.matrix(np.hstack((h_1, h_2)))

        # solve QP problem
        solution = cvxopt.solvers.qp(P, q, G=G, h=h, A=A, b=b)

        # Lagrange multipliers
        alpha = np.array(solution['x']).squeeze()

        # Support vectors have non zero lagrange multipliers
        sv = np.where(np.logical_and(alpha != 0, alpha != self.C)) if self.C else \
        np.where(alpha != 0)

        #this will actually give the indices of the support vectors

        # get alphas of support vector , Xs and ys too.
        self.alpha = alpha[sv]
        self.sv = X[sv]
        self.sv_y = y[sv]


        # Intercept
        self.b = 1 / self.sv_y[0]
        for i in range(len(self.alpha) ):
            self.b -= self.alpha[i] * self.sv_y[i] * self.kernel(self.sv[i], self.sv[0])

        # Weight vector
        if self.kernel == 'linear':
            self.w = np.zeros(n_features)
            for i in range(len(self.alpha) ):
                self.w += self.alpha[i] * self.sv_y[i] * self.sv[i]
        else:
            self.w = None #Guidance: for non-linear case this should be None. (do not change)

    def predict(self, X):
        if self.w is not None:
            return np.sign(np.dot(X, self.w) + self.b)
        else:

            y_predict = np.zeros(len(X))
            for i in range(len(X)):
                for j in range(len(self.alpha)):
                  y_predict[i] += self.sv_y[j] * self.alpha[j] * self.kernel(X[i], self.sv[j])
            return np.sign(y_predict + self.b)



In [41]:
import numpy as np
import cvxopt

class SVM:
    def __init__(self, kernel='linear', C=1.0):
        """Initialize the SVM model with kernel and regularization parameters.

        Parameters
        ----------
        kernel : str, optional
            The kernel function to use. Can be 'linear', 'poly', 'rbf' or 'sigmoid'. Default is 'linear'.

        C : float, optional
            The regularization parameter. A smaller value of C allows more misclassification. Default is 1.0.
        """

        if kernel == 'linear':
            self.kernel_name = 'linear'
            self.kernel = lambda x1, x2: np.dot(x1, x2)
        elif kernel == 'poly':
            self.kernel_name = 'poly'
            self.kernel = lambda x1, x2: (np.dot(x1, x2) + 1)**2
        elif kernel == 'rbf':
            self.kernel_name = 'rbf'
            self.kernel = lambda x1, x2: np.exp(-np.linalg.norm(x1 - x2)**2)
        elif kernel == 'sigmoid':
            self.kernel_name = 'sigmoid'
            self.kernel = lambda x1, x2: np.tanh(np.dot(x1, x2))
        else:
            raise ValueError('Invalid kernel')


        # Define regularization parameter
        self.C = C
        self.binary_classifiers = []

    def fit(self, X, y):
        """Fit the SVM model to the dataset using one-vs-one approach.

        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training data, where `n_samples` is the number of samples
            and `n_features` is the number of features.

        y : array-like of shape (n_samples,)
            Training labels, where `n_samples` is the number of samples.

        Returns
        -------
        self : object
            Returns the instance itself.
        """

        # Get number of samples and features
        n_samples, n_features = X.shape

        # Get unique classes
        classes = np.unique(y)
        n_classes = len(classes)

        # Train one-vs-one classifiers
        for i in range(n_classes):
            for j in range(i + 1, n_classes):
                # Select samples corresponding to the two classes
                class_i_indices = np.where(y == classes[i])[0]
                class_j_indices = np.where(y == classes[j])[0]
                indices = np.concatenate((class_i_indices, class_j_indices))
                X_ij = X[indices]
                y_ij = y[indices]

                # Convert labels to -1 and 1
                y_ij[y_ij == classes[i]] = -1
                y_ij[y_ij == classes[j]] = 1

                # Train binary classifier
                classifier =  SVM_bi(self.C, kernel=self.kernel_name)#SVC(kernel=self.kernel_name, C=self.C)
                classifier.fit(X_ij, y_ij)
                self.binary_classifiers.append((classes[i], classes[j], classifier))

        return self

    def predict(self, X):
        """Predict the class labels for the test data X.

        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Test data, where `n_samples` is the number of samples
            and `n_features` is the number of features.

        Returns
        -------
        y_pred : array-like of shape (n_samples,)
            Predicted labels, where `n_samples` is the number of samples.
        """

        # Initialize votes for each class
        n_samples = X.shape[0]
        n_classes = len(self.binary_classifiers)#len(np.unique([label for label in self.binary_classifiers]))

        votes = np.zeros((n_samples, n_classes))

        # Make predictions with each binary classifier
        for class_i, class_j, classifier in self.binary_classifiers:
            class_i_indices = np.where(classifier.predict(X) == -1)[0]
            class_j_indices = np.where(classifier.predict(X) == 1)[0]
            votes[class_i_indices, class_i] += 1
            votes[class_j_indices, class_j] += 1

        # Determine the class with the most votes
        y_pred = np.argmax(votes, axis=1)

        return y_pred



Report the accuracy and F1-score of the classifier on the testing set **(P2-2-2: 10 points)**

In [42]:
from sklearn.metrics import accuracy_score, f1_score
svm = SVM(kernel='linear')

# Fit the SVM model to the training data
svm.fit(x_train, y_train)
y_pred_test = svm.predict(x_test)
accuracy = accuracy_score(y_test, y_pred_test)
f1 = f1_score(y_test, y_pred_test, average='weighted')

print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test F1-score: {f1:.4f}")

Streaming output truncated to the last 5000 lines.
 6: -1.8704e-02 -1.8715e-02  1e-05  5e-11  5e-15
 7: -1.8709e-02 -1.8710e-02  5e-07  4e-13  4e-15
 8: -1.8709e-02 -1.8709e-02  1e-08  4e-15  3e-15
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -4.2908e-03 -1.6568e+01  7e+01  1e+00  2e-14
 1:  5.2520e-02 -4.9060e+00  6e+00  4e-02  2e-14
 2:  1.4030e-02 -2.2221e-01  2e-01  5e-04  2e-14
 3: -2.9613e-02 -5.1602e-02  2e-02  2e-06  1e-14
 4: -3.2630e-02 -3.4836e-02  2e-03  1e-07  9e-15
 5: -3.3096e-02 -3.3429e-02  3e-04  1e-08  7e-15
 6: -3.3206e-02 -3.3229e-02  2e-05  1e-10  8e-15
 7: -3.3217e-02 -3.3218e-02  7e-07  1e-12  1e-14
 8: -3.3218e-02 -3.3218e-02  7e-09  1e-14  8e-15
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -3.5615e-03 -1.6464e+01  7e+01  1e+00  1e-14
 1:  5.2162e-02 -4.6991e+00  5e+00  3e-02  2e-14
 2:  2.0918e-03 -1.8812e-01  2e-01  4e-04  1e-14
 3: -3.0048e-02 -4.6564e-02  2e-02  2e-06  1e-14
 4: -3.2061e-02 -3.37

# Sec 3: PCA

Fetch all images taken from `jones`, `taylor`, `anderson` and `wilson` (use the `labelencoder` object). Perform PCA from scratch using the following `class` to reduce the dimension of those images to 3. Save the results as a `numpy` array of shape `(40, 3)`. **(P3-1: 30 points)**

In [60]:
import numpy as np

class PCA:
    def __init__(self, n_components):
        self.n_components = n_components
        self.components_ = None

    def fit(self, X):
        """Fit the model with X.

        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training data, where `n_samples` is the number of samples
            and `n_features` is the number of features.

        Returns
        -------
        self : object
            Returns the instance itself.
        """
        self.components_ = None

        # Center the data
        X_centered = X - np.mean(X, axis=0)

        # Compute the covariance matrix
        covariance = np.cov(X_centered, rowvar=False)

        # Compute the eigenvalues and eigenvectors of the covariance matrix
        eigenvalues, eigenvectors = np.linalg.eig(covariance)

        # Sort the eigenvalues and eigenvectors in descending order
        sorted_indices = np.argsort(eigenvalues)[::-1]
        eigenvalues = eigenvalues[sorted_indices]
        eigenvectors = eigenvectors[:, sorted_indices]

        # Select the top n_components eigenvectors as the principal components
        self.components_ = eigenvectors[:, :self.n_components]

        return self

    def transform(self, X):
        """Fit the model with X and apply the dimensionality reduction on X.

        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training data, where `n_samples` is the number of samples
            and `n_features` is the number of features.

        Returns
        -------
        X_new : ndarray of shape (n_samples, n_components)
            Transformed values.
        """
        assert self.components_ is not None

        # Center the data
        X_centered = X - np.mean(X, axis=0)

        # Project the centered data onto the principal components
        X_new = np.dot(X_centered, self.components_)

        return X_new


In [61]:
target_names = ['jones', 'taylor', 'anderson', 'wilson']
target_indices = label_encoder.transform(target_names)

filtered_images = x[np.isin(y, target_indices)]
filtered_labels = y[np.isin(y, target_indices)]
filtered_images = filtered_images.reshape(filtered_images.shape[0], -1)

pca = PCA(n_components=3)
pca.fit(filtered_images)


In [62]:
pca_result = pca.transform(filtered_images)

Use `plotly` to plot these 3D samples. You should use their labels as the determiner of their color. **(P3-2: 10 points)**

In [63]:
pca_result_real = np.real(pca_result)

In [64]:
pca_df = pd.DataFrame({'PC1': pca_result_real[:, 0], 'PC2': pca_result_real[:, 1], 'PC3': pca_result_real[:, 2], 'Label': filtered_labels})

# Plot PCA results in 3D
fig = px.scatter_3d(pca_df, x='PC1', y='PC2', z='PC3', color='Label', symbol='Label', hover_data=['Label'])
fig.show()

In [67]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Scatter3d(
    x=pca_result_real[:, 0],
    y=pca_result_real[:, 1],
    z=pca_result_real[:, 2],
    mode='markers',
    marker=dict(
        size=5,
        colorscale='Viridis',
        opacity=0.8
    )
)])

fig.update_layout(
    scene=dict(
        xaxis_title='PC1',
        yaxis_title='PC2',
        zaxis_title='PC3'
    ),
)

fig.show()


# Sec 4: K-Means

Using K-means, we want to perform clustering on the 3D samples. Consider 4 clusters. After running the K-means algorithm on the data, determine to which cluster each sample belongs. Then, use `plotly` to plot the 3D samples colored with respect to their clusters. **(P4-1: 15 points)**

In [68]:
import numpy as np

class KMeans:
    """
    Fits it on data, then uses predict to get cluster labels.
    """

    def __init__(self, n_clusters=4, max_iter=100):
        """
        Initialize KMeans

        Parameters
        ----------
        n_clusters : int, optional (default=4)
            The number of clusters to form.

        max_iter : int, optional (default=100)
            The maximum number of iterations to perform.
        """
        self.n_clusters = n_clusters
        self.max_iter = max_iter

    def fit(self, x, y=None):
        """Fits the training data.

        Parameters
        ----------
        x : array-like, shape = [n_samples, n_features]
            Training samples

        y : Not used

        Returns
        -------
        self : object
        """
        n_samples, n_features = x.shape

        # Randomly initialize cluster centers
        random_indices = np.random.choice(n_samples, self.n_clusters, replace=False)
        self.cluster_centers_ = x[random_indices]

        # Assign initial labels
        labels = self._assign_labels(x)

        # Iterate until convergence or maximum iterations
        for _ in range(self.max_iter):
            old_centers = self.cluster_centers_

            # Update cluster centers
            self._update_centers(x, labels)

            # Assign new labels
            labels = self._assign_labels(x)

            # Check convergence
            if np.all(old_centers == self.cluster_centers_):
                break

        return self

    def predict(self, x):
        """Predicts the cluster labels.

        Parameters
        ----------
        x : array-like, shape = [n_samples, n_features]
            Test samples

        Returns
        -------
        Predicted cluster labels
        """
        return self._assign_labels(x)

    def _assign_labels(self, x):
        """Assigns labels to samples based on current cluster centers.

        Parameters
        ----------
        x : array-like, shape = [n_samples, n_features]
            Samples

        Returns
        -------
        Cluster labels
        """
        distances = self._compute_distances(x)
        labels = np.argmin(distances, axis=1)
        return labels

    def _compute_distances(self, x):
        """Computes distances between samples and cluster centers.

        Parameters
        ----------
        x : array-like, shape = [n_samples, n_features]
            Samples

        Returns
        -------
        Distances between samples and cluster centers
        """
        distances = np.zeros((x.shape[0], self.n_clusters))
        for i in range(self.n_clusters):
            distances[:, i] = np.linalg.norm(x - self.cluster_centers_[i], axis=1)
        return distances

    def _update_centers(self, x, labels):
        """Updates cluster centers based on current assignments.

        Parameters
        ----------
        x : array-like, shape = [n_samples, n_features]
            Samples

        labels : array-like, shape = [n_samples]
            Current cluster assignments
        """
        for i in range(self.n_clusters):
            cluster_samples = x[labels == i]
            self.cluster_centers_[i] = np.mean(cluster_samples, axis=0)


In [78]:
import numpy as np
import plotly.graph_objects as go

# Convert complex numbers to real numbers
pca_result_real = np.real(pca_result)

# Perform K-means clustering
kmeans = KMeans(n_clusters=4)
kmeans.fit(pca_result_real)
cluster_labels = kmeans.predict(pca_result_real)

# Plot 3D samples colored with respect to their clusters
fig = go.Figure(data=[go.Scatter3d(
    x=pca_result_real[:, 0],
    y=pca_result_real[:, 1],
    z=pca_result_real[:, 2],
    mode='markers',
    marker=dict(
        size=5,
        color=cluster_labels,  # Use cluster labels for color mapping
        colorscale='Viridis',
        opacity=0.8
    )
)])

fig.update_layout(
    scene=dict(
        xaxis_title='PC1',
        yaxis_title='PC2',
        zaxis_title='PC3'
    ),
    title='K-means Clustering (4 Clusters)'
)

fig.show()


Measure the quality of the clustering using "silhoutte score" **(P4-2: 5 points)**

Note: You can use `sklearn` or other packages to calculate the score.

In [79]:
from sklearn.metrics import silhouette_score

silhouette_avg = silhouette_score(pca_result_real, cluster_labels)

print("Silhouette Score:", silhouette_avg)


Silhouette Score: 0.33791389363219715


Compare the result of the clustering with the original labels. To be more precise, in this part, you should clearly answer the following questions with details:<br>

- Does each cluster represent a true label?
- What is your conclusion?

**(P4-3: 10 points)**

A silhouette score of 0.337 suggests a moderate level of clustering quality. Generally, a silhouette score above 0.5 is considered to indicate a good clustering structure, while scores below 0.2 suggest weak or overlapping clusters. In this case, the silhouette score of 0.337 indicates that the clustering has some degree of separation and cohesion among the samples, but there is still room for improvement. It's important to note that the interpretation of the silhouette score may vary depending on the specific dataset and context.

it can be better if we use this function and run it multiple times to get better accuraccy because this function runs with using random samples and by repeating you can check different randon samples that are chosen and get different results.

and about representing the label, the data is very complex and vary in a large domain its very hard to find a good clustering and for some samples the clustering works well but for some of them its does not work good because some datapoints are closer to other clusters.

In [102]:
metadata = pd.read_csv('meta.csv')

In [103]:
img_paths = metadata['path'].values
images = []
for img_path in img_paths:
    img = Image.open('content/img-data/' +img_path).convert('L')
    img = np.array(img)
    images.append(img)
    img = img.resize((64, 64))
x = np.array(images)


In [104]:
x = x / 255


In [105]:
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(metadata['label'])

In [127]:
random_state = 42
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify=y, random_state=random_state)

# Sec 5: Neural Net (Optional with Bonus Points)

Reshape the images so that they become of shape `64x64` **(P5-1: 5 points)**

In [ ]:
# this was done when loading images.

Use different types of augmentation techniques (such as Random Flipping, Random Rotation, etc.) to increase the size of the training dataset. **(P5-2: 10 points)**

In [128]:
import random

flip_probability = 0.5  # Probability of flipping the image
rotation_angles = [0, 30, 60, 90, 120, 150, 180, 210, 240, 270, 300, 330]  # Possible rotation angles

# Create empty lists for augmented images and labels
augmented_images_train = []
train_labels = []
# Iterate over each image and perform random image augmentation
for j in range(3):
  for i in range(len(x_train)):
      image = x_train[i]
      label = y_train[i]

      # Append original image and label
      augmented_images_train.append(image)
      train_labels.append(label)

      # Randomly select augmentation parameters
      flip = random.random() < flip_probability
      rotation_angle = random.choice(rotation_angles)

      # Perform image augmentation based on the selected parameters
      # 1. Flip horizontally

      if flip:
          print(image)
          image = np.fliplr(image)
          print(image)
          augmented_images_train.append(image)
          train_labels.append(label)

      # 2. Rotate by a random angle
      image_new = np.rot90(image, k=rotation_angle // 90)
      augmented_images_train.append(image_new)
      train_labels.append(label)

# Convert the augmented image and label lists to numpy arrays
augmented_images_train = np.array(augmented_images_train)
train_labels = np.array(train_labels)


ValueError: ignored

In [100]:
augmented_images_test = []
test_labels = []

# Iterate over each image and perform random image augmentation
for i in range(len(x_test)):
    image = x_test[i]
    label = y_test[i]

    # Append original image and label
    augmented_images_test.append(image)
    test_labels.append(label)

    # Randomly select augmentation parameters
    flip = random.random() < flip_probability
    rotation_angle = random.choice(rotation_angles)

    # Perform image augmentation based on the selected parameters
    # 1. Flip horizontally
    if flip:
        image = np.fliplr(image)
        augmented_images_test.append(image)
        test_labels.append(label)

    # 2. Rotate by a random angle
    image_new = np.rot90(image, k=rotation_angle // 90)
    augmented_images_test.append(image_new)
    test_labels.append(label)

# Convert the augmented image and label lists to numpy arrays
augmented_images_test = np.array(augmented_images_test)
y_test = np.array(test_labels)


Using the augmented training set, train a neural network consisting of 2D convolutional layers. The exact architecture of the network is arbitrary. However, the last layer should have the "softmax" function as its activation. Also, using "max pooling" layers after convolutional ones is advised.

Note: Don't get frustrated if your network does not appear to be a good one. Give it some time to be trained. For instance, you might need to train it for 100 epochs. Make sure to try that out before quitting!

**(P5-3: 20 points)**

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

# Create the CNN model
#preprocessed_x_train = augmented_images_train.reshape(-1, 64, 64, 3)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(64, 64, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(39, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(augmented_images_train, train_labels, batch_size=64, epochs=20, validation_split=0.2, verbose=1)


Epoch 1/20
30/30 [==============================] - 13s 397ms/step - loss: 3.6648 - accuracy: 0.0469 - val_loss: 3.6363 - val_accuracy: 0.0458
Epoch 2/20
30/30 [==============================] - 12s 394ms/step - loss: 3.5131 - accuracy: 0.0788 - val_loss: 3.3172 - val_accuracy: 0.2104
Epoch 3/20
30/30 [==============================] - 12s 395ms/step - loss: 2.8036 - accuracy: 0.2968 - val_loss: 2.3167 - val_accuracy: 0.3708
Epoch 4/20
30/30 [==============================] - 13s 445ms/step - loss: 1.6650 - accuracy: 0.5994 - val_loss: 1.4764 - val_accuracy: 0.6250
Epoch 5/20
30/30 [==============================] - 12s 391ms/step - loss: 1.0104 - accuracy: 0.7402 - val_loss: 0.9641 - val_accuracy: 0.7500
Epoch 6/20
30/30 [==============================] - 12s 395ms/step - loss: 0.6541 - accuracy: 0.8341 - val_loss: 0.6918 - val_accuracy: 0.8250
Epoch 7/20
30/30 [==============================] - 12s 409ms/step - loss: 0.4459 - accuracy: 0.8967 - val_loss: 0.5196 - val_accuracy: 0.8625

Measure the accuracy of the network on the validation and testing datasets **(P5-4: 5 points)**

In [93]:
# Evaluate the model on the testing set
loss, accuracy = model.evaluate(np.array(augmented_images_test).reshape(-1, 64, 64, 1), y_test)
print('Test Loss:', loss)
print('Test Accuracy:', accuracy)


7/7 [==============================] - 0s 40ms/step - loss: 0.7491 - accuracy: 0.8469
Test Loss: 0.7491307258605957
Test Accuracy: 0.8469387888908386
